<a href="https://colab.research.google.com/github/pranavkantgaur/Coursera_DL_specialization_from_scratch/blob/master/course1/week4/build_nn_step_by_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objectives
To be able to build and train NNs for various depth and width. This assignment is not tied to any application, therefore it will be evaluated using test cases. The functions developed in this notebook will be used in the next assignment. Target is to build:


*   A 2-layer NN
*   A L-layer NN

Effectively, I will be able to build and train a L-layer fully connected NN, entirely using Numpy.


In [0]:
class TwoLayerNN(object):
  def __init__(self):
    # hyperparameters
    self.n_hidden_units = n_hidden_units # len(self.n_hidden_units = number of layers)
    self.n_epochs = 0
    self.lr = 0
    # parameters
    self.w1 = 0
    self.b1 = 0
    self.w2 = 0
    self.b2 = 0
    self.w3 = 0
    self.b3 = 0

  def forward(self, X):
    # layer 1
    z1 = np.dot(self.w1, X) + self.b1
    a1 = np.tanh(z1)
    
    # layer 2
    z2 = np.dot(self.w2, a1) + self.b2
    a2 = np.tanh(z2)
    
    # layer 3 (output)
    z3 = np.dot(self.w3, a2) + self.b3
    a3 = sigmoid(z3)

    return a1, a2, a3

  def initialize_parameters(self):
    self.w1 = np.random.randn(n_hidden_units[0], input_size) * 0.01
    self.b1 = np.zeros(n_hidden_units[0], 1)
    self.w2 = np.random.randn(n_hidden_units[1], num_hidden_units[0]) * 0.01
    self.b2 = np.zeros(n_hidden_units[1], 1)
    self.w3 = np.random.randn(n_hidden_units[2], num_hidden_units[1]) * 0.01
    self.b3 = np.zeros(n_hidden_units[2], 1)


  def backward(self):
    '''
    Notice that the only difference across layers lies in computation of dZ, 
    rest all can be parameterized on layer ID.
    '''
    dZ3 = A3 - Y # output layer
    dw3 = np.dot(dZ3, A2.T) / m
    db3 = np.mean(dZ3)
    
    dZ2 = np.dot(self.w2.T, dZ3) * (1 - np.power(A2, 2))
    dw2 = np.dot(dZ2, A1.T) / m
    db2 = np.mean(dZ2)

    dZ1 = np.dot(self.w1.T, dZ2) * (1 - np.power(A1, 1))
    dw1 = np.dot(dZ1, X.T) / m
    db1 = np.mean(dZ1)

    return dw1, db1, dw2, db2, dw3, db3

  def update_parameters(self):
    None
  def compute_cost(self):
    None
  def train(self):
    None
  def evaluate(self):
    None              
   

In [0]:
twoNN = TwoLayerNN()

In [0]:
twoNN.train()

In [0]:
twoNN.evaluate()